**mode collapse:** generator stuck in a single mode (peak of the distribution)

**loss value meaning:** not a good correlation between the loss value and the quality of the generated results

**bce loss problems -** flat gradients

we'll use **the wasserstein loss** instead of binary cross entropy.

**earth mover distance:** the effort needed to make both real and generated distributions equal.

In [1]:
# coding an advanced GAN from scratch in python + pytorch
# importing the libraries

import torch, torchvision, os, PIL, pdb # pytorch and torchvision are for datasets and image transforms, os if for file handling, pil is for image processing, and pdb if for debugging
from torch import nn # neural network module
from torch.utils.data import Dataset # dataset class and data loader to create custom datasets and load them in batches
from torch.utils.data import DataLoader
from torchvision import transforms # transforms module from torchvision to apply image preprocessing
from torchvision.utils import make_grid # function to create a grid of images for visualisation
from tqdm.auto import tqdm # progress bar tool for loops
import matplotlib.pyplot as plt # for plotting images and graphs
import numpy as np # for numerical operations
from PIL import Image # image module from pillow for image loading and handling

In [2]:
def show(tensor, num = 25, wandb = 0, name = ''):
  data = tensor.detach().cpu()   # detach tensor from computation graph and move it to cpu
  grid = make_grid(data[:num], nrow = 5).permute(1, 2, 0)   # create a grid of images (5 per row), rearrange axes for plotting

  # optional: log images to wandb if enabled
  if (wandb == 1 and wandbact ==1):
    wandb.log({name: wandb.Image(grid.numpy().clip(0,1))})

  plt.imshow(grid.clip(0, 1))
  plt.show()

  #hyperparameters and general parameters
  n_epochs = 100000 # number of training iterations
  batch_size = 128 # number of images per batch
  lr = 1e-4 # learning rate for optimiser
  z_dim = 200 # size of the noise vector for generator
  device = 'cuda' # GPU for training

  cur_step = 0 # current step in training
  crit_cycles = 5 # number of critic updates per generator update (used in wgan)
  gen_losses = [] # to track generator loss over time
  crit_losses = [] # to track critic/discriminator loss over time
  show_step = 35 # how often to display generated image
  save_step = 35 # how often to display model check points

  wandbact = 1 # yes we want to track stats through weights and biases, optional

In [5]:
# optional
!pip install wandb
import wandb
wandb.login(key = 'YOUR KEY HERE')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: burciadilammermoore (burciadilammermoore-middle-east-technical-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
%%capture
experiment_name = wandb.util.generate_id() # generate a unique id for the experiment

myrun = wandb.init( # initialise a wandb run
    project = 'wgan', # name of the project: wassersteing gan
    group = experiment_name, # group name used to group related runs
    config = { # configuration dictionary
        'optimizer': 'sgd', # use stochastic gradient descent
        'model': 'wgan gp', # model type is wasserstein gan with gradient penalty
        'epoch': '1000', # traning epochs
        'batch_size': 128 # samples per batch
    }
)

config = wandb.config # store wand config for early access

In [7]:
print(experiment_name)

zkujyczk
